In [741]:
A = [[1, 0, 0, 1, 0],[1, 0, 1, 0, 1], [0, 1, 1, 0, 1], [0, 1, 0, 0, 1], [1, 1, 1, 0, 1]]
U = [[1.9 ,2.5 ,1.05], [2.2, 4.2, 0.9], [4.1, 0.36, 3.1], [5.0, 3.1, 2.36],[1.2, 7.002, 6.3]]
V = [[1.2, 6.21, 1.09, 8.01, 2.36], [2.1, 7.0, 6.36, 0.63, 1.01], [1.002, 9, 6.2, 5.8, 0.05]]
X=[[2,0.63,1.2,1.2],[2.1,0.1,0.6,0.3],[6.3,1.2,5.6,0.19]]
print("A=",A)
print("U=",U)
print("V=",V)
print("X=",X)

A= [[1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [0, 1, 1, 0, 1], [0, 1, 0, 0, 1], [1, 1, 1, 0, 1]]
U= [[1.9, 2.5, 1.05], [2.2, 4.2, 0.9], [4.1, 0.36, 3.1], [5.0, 3.1, 2.36], [1.2, 7.002, 6.3]]
V= [[1.2, 6.21, 1.09, 8.01, 2.36], [2.1, 7.0, 6.36, 0.63, 1.01], [1.002, 9, 6.2, 5.8, 0.05]]
X= [[2, 0.63, 1.2, 1.2], [2.1, 0.1, 0.6, 0.3], [6.3, 1.2, 5.6, 0.19]]


In [742]:
import numpy as np
A=np.array(A)
U=np.array(U)
V=np.array(V)
X=np.array(X)
print("A=\n",A)
print("U=\n",U)
print("V=\n",V)
print("X=\n",X)

A=
 [[1 0 0 1 0]
 [1 0 1 0 1]
 [0 1 1 0 1]
 [0 1 0 0 1]
 [1 1 1 0 1]]
U=
 [[1.9   2.5   1.05 ]
 [2.2   4.2   0.9  ]
 [4.1   0.36  3.1  ]
 [5.    3.1   2.36 ]
 [1.2   7.002 6.3  ]]
V=
 [[1.2   6.21  1.09  8.01  2.36 ]
 [2.1   7.    6.36  0.63  1.01 ]
 [1.002 9.    6.2   5.8   0.05 ]]
X=
 [[2.   0.63 1.2  1.2 ]
 [2.1  0.1  0.6  0.3 ]
 [6.3  1.2  5.6  0.19]]


# integrality_gap_elastic

In [535]:
def integrality_gap_elastic(e, kappa, lambda_):
    return min(kappa * abs(e) + lambda_ * e**2, kappa * abs(e - 1) + lambda_ * (e - 1)**2)


In [536]:
integrality_gap_elastic(2,0.01,0.02)

0.03

# regularizer_elbmf

In [537]:
def regularizer_elbmf(x, l1reg, l2reg):
    s=0
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            s+=(integrality_gap_elastic(x[i][j], l1reg, l2reg))
    return s


In [538]:
regularizer_elbmf(X,0.001,0.002)

0.117986

# proxel_1

In [539]:
def proxel_1(x, k, l):
    return (x - k * np.sign(x) if x <= 0.5 else (x - k * np.sign(x - 1) + l)) / (1 + l)

In [540]:
proxel_1(4,0.1,0.3)

3.230769230769231

# proxelp

In [541]:
def proxelp(x, k, l):
    return np.maximum(proxel_1(x, k, l), np.zeros_like(x))


In [542]:
proxelp(3, 0.1, 0.2)

2.5833333333333335

# prox_elbmf

In [543]:
def prox_elbmf(X, k, l):
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            X[i][j] = proxelp(X[i][j], k, l)


In [544]:
prox_elbmf(X, 0.01, 0.02)

In [545]:
print(X)

[[1.97058824 0.64705882 1.18627451 1.18627451]
 [2.06862745 0.08823529 0.61764706 0.28431373]
 [6.18627451 1.18627451 5.5        0.17647059]]


# proxelb

In [546]:
def proxelb(x, k, l):
    return np.clip(proxel_1(x, k, l), 0, 1)


In [547]:
proxelb(2,0.01,0.02)

1.0

# prox_elbmf_box

In [548]:
def prox_elbmf_box(X, k, l):
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            X[i][j] = proxelb(X[i][j], k, l)


In [549]:
prox_elbmf_box(X,0.0,0.02)

In [550]:
print(X)

[[1.         0.65397924 1.         1.        ]
 [1.         0.08650519 0.62514418 0.27873895]
 [1.         1.         1.         0.17301038]]


# structs

In [551]:
class PALM:
    pass

class iPALM:
    def __init__(self, beta):
        self.beta = beta


In [552]:
p = PALM()
ip = iPALM(0.2)

In [553]:
class ElasticBMF:
    def __init__(self,l1reg,l2reg):
        self.l1reg=l1reg
        self.l2reg=l2reg

In [554]:
fn = ElasticBMF(0.001,0.002)

# rounding

In [555]:
def rounding(fn, _, U: np.ndarray, V: np.ndarray) -> None:
    prox_elbmf(U, 0.5, 1e20)
    np.clip(U,0,1)
    np.round(U)
    
    prox_elbmf(V, 0.5, 1e20)
    np.clip(V,0,1)
    np.round(V)
    


In [556]:
U

array([[1.9  , 2.5  , 1.05 ],
       [2.2  , 4.2  , 0.9  ],
       [4.1  , 0.36 , 3.1  ],
       [5.   , 3.1  , 2.36 ],
       [1.2  , 7.002, 6.3  ]])

In [557]:
V

array([[1.2  , 6.21 , 1.09 , 8.01 , 2.36 ],
       [2.1  , 7.   , 6.36 , 0.63 , 1.01 ],
       [1.002, 9.   , 6.2  , 5.8  , 0.05 ]])

In [558]:
# rounding(fn,A,U,V)

In [559]:
U

array([[1.9  , 2.5  , 1.05 ],
       [2.2  , 4.2  , 0.9  ],
       [4.1  , 0.36 , 3.1  ],
       [5.   , 3.1  , 2.36 ],
       [1.2  , 7.002, 6.3  ]])

In [560]:
V

array([[1.2  , 6.21 , 1.09 , 8.01 , 2.36 ],
       [2.1  , 7.   , 6.36 , 0.63 , 1.01 ],
       [1.002, 9.   , 6.2  , 5.8  , 0.05 ]])

# apply rate

In [561]:
def apply_rate(fn, fn0, nu):
    fn.l2reg = fn0.l2reg * nu


In [562]:
apply_rate(fn,fn,2)

In [563]:
fn.l1reg

0.001

In [564]:
fn.l2reg

0.004

# PALM reducemf_impl with PALM

In [719]:
def reducemf_impl(prox,p:PALM,A, U, V):
    VVt = V @ V.T
    AVt = A @ V.T
    L = max(np.linalg.norm(VVt), 1e-4)
    step_size = 1 / (1.1 * L)
    grad = U @ VVt - AVt
    a,b = U.shape
    for i in range(a):
        for j in range(b):
            U[i][j] -= grad[i][j] * step_size
    prox(U, step_size)


In [720]:
# reducemf_impl(p,A,U,V)

In [721]:
U

array([[1.9  , 2.5  , 1.05 ],
       [2.2  , 4.2  , 0.9  ],
       [4.1  , 0.36 , 3.1  ],
       [5.   , 3.1  , 2.36 ],
       [1.2  , 7.002, 6.3  ]])

# PALM reducemf_impl with iPALM

In [692]:
import copy

In [722]:
def reducemf_impl_ip(prox,opt:iPALM,A, U, V,U_):
    VVt = V @ V.T
    AVt = A @ V.T
    
    
    L = max(np.linalg.norm(VVt), 1e-4)
   
    
    a,b = U.shape
    
    #@. U = U + opt.beta * (U - U_)
    for i in range(a):
        for j in range(b):
            U[i][j]= U[i][j]+opt.beta*(U[i][j]-U_[i][j])
            
    #@. U_ = U
    for i in range(a):
        for j in range(b):
            U_[i][j] = U[i][j]
            
    step_size = 2*(1-opt.beta) / (1+2*opt.beta)/L
    
    grad = U @ VVt - AVt
    for i in range(a):
        for j in range(b):
            U[i][j] = U[i][j]-grad[i][j] * step_size
    prox(U, step_size)




In [694]:
# ip=iPALM(0.4)

In [695]:
# U_=copy.deepcopy(U)

In [696]:
# U

In [697]:
# U_

In [698]:
# reducemf_impl_ip(ip,A, U, V,U_)

In [699]:
# U

In [700]:
# U_

# reduce emf!

In [745]:
def reducemf(fn, opt:PALM, A, U, V):
    def prox(x, alpha):
        return prox_elbmf(x, fn.l1reg * alpha, fn.l2reg * alpha)
    reducemf_impl(prox, opt, A, U, V)

# reduce emf! iPALM

In [746]:
def reducemf_i(fn, opt:iPALM, A, U, V,U_):
    def prox(x, alpha):
        return prox_elbmf(x, fn.l1reg * alpha, fn.l2reg * alpha)
    reducemf_impl_ip(prox, opt, A, U, V,U_)
  

In [744]:
#before
print("A=\n",A)
print("U=\n",U)
print("V=\n",V)
print("UV=\n",np.dot(U,V))

ip = iPALM(0.2)
U_=np.copy(U_)
reducemf_i(fn, ip, A, U, V,U_)

#after
print("A=\n",A)
print("U=\n",U)
print("V=\n",V)
print("UV=\n",np.dot(U,V))

A=
 [[1 0 0 1 0]
 [1 0 1 0 1]
 [0 1 1 0 1]
 [0 1 0 0 1]
 [1 1 1 0 1]]
U=
 [[1.9   2.5   1.05 ]
 [2.2   4.2   0.9  ]
 [4.1   0.36  3.1  ]
 [5.    3.1   2.36 ]
 [1.2   7.002 6.3  ]]
V=
 [[1.2   6.21  1.09  8.01  2.36 ]
 [2.1   7.    6.36  0.63  1.01 ]
 [1.002 9.    6.2   5.8   0.05 ]]
UV=
 [[  8.5821   38.749    24.481    22.884     7.0615 ]
 [ 12.3618   51.162    34.69     25.488     9.479  ]
 [  8.7822   55.881    25.9786   51.0478   10.1946 ]
 [ 14.87472  73.99     39.798    55.691    15.049  ]
 [ 22.4568  113.166    84.90072  50.56326  10.21902]]
A=
 [[1 0 0 1 0]
 [1 0 1 0 1]
 [0 1 1 0 1]
 [0 1 0 0 1]
 [1 1 1 0 1]]
U=
 [[0.19816411 0.81044163 0.        ]
 [0.04119106 1.92732297 0.        ]
 [1.15798984 0.         0.        ]
 [1.38649508 0.03699553 0.        ]
 [0.         1.90108588 0.        ]]
V=
 [[1.2   6.21  1.09  8.01  2.36 ]
 [2.1   7.    6.36  0.63  1.01 ]
 [1.002 9.    6.2   5.8   0.05 ]]
UV=
 [[ 1.93972435  6.9036905   5.37040764  2.09787271  1.28621333]
 [ 4.09680751 13.7

# factorize_palm

In [723]:
import copy
def factorize_palm(fn, A, U, V, regularization_rate, maxiter, tol, callback=None):
    ell = float("inf")
    fn0 = copy.deepcopy(fn)
    for t in range(1,maxiter):
        fn.l2reg = fn0.l2reg * regularization_rate
        reducemf(fn, PALM,A, U, V)
        reducemf(fn, PALM,A.T, V.T, U.T)
#         ell, ell0 = np.linalg.norm(X - U @ V)**2, ell
#         ell0 = ell
#         ell = np.linalg.norm(X - U @ V)**2
#         if callback:
#             callback((U, V), ell)
#         if abs(ell - ell0) < tol:
#             break
    fn = fn0
    return U, V



In [713]:
def factorize_ipalm(
    fn,
    A,
    U,
    V,
    regularization_rate,
    maxiter,
    tol,
    beta,
    callback=None
):
    if beta == 0:
        return factorize_palm(fn, A, U, V, regularization_rate, maxiter, tol, callback=callback)

    ell = float("inf")
    fn0 = copy.deepcopy(fn)

    ip = iPALM(beta)
    U_ = np.copy(U)
    Vt_ = np.copy(V.T)

    for t in range(1,maxiter):
        fn.l2reg = fn0.l2reg * regularization_rate

        reducemf_i(fn, ip, A, U, V, U_)
        reducemf_i(fn, ip, A.T, V.T, U.T, Vt_)

#         ell0 = ell
#         ell = np.sum((A - np.dot(U, V)) ** 2)

#         if callback is not None:
#             callback((U, V), ell)
#         if abs(ell - ell0) < tol:
#             break
    fn = fn0
    return U, V


# Calling PALM

In [728]:

n,m=A.shape
print(n,m)
k=4
# U = np.random.randint(2,size=(n, k)).astype("float")
# V = np.random.randint(2,size=(k, m)).astype("float")
U,V=factorize_palm(fn,A,U,V,0.01,1000,0.01)
# rounding(fn,A,U,V)


5 5


In [729]:
print(U.shape,V.shape)
print("A=\n",A)
print("U=\n",U)
print("V=\n",V)
print("UV=\n",np.dot(U,V))

(5, 3) (3, 5)
A=
 [[1 0 0 1 0]
 [1 0 1 0 1]
 [0 1 1 0 1]
 [0 1 0 0 1]
 [1 1 1 0 1]]
U=
 [[0.00000000e+00 1.57305208e+00 8.59050809e-02]
 [3.41482880e-01 2.82389151e+00 0.00000000e+00]
 [9.86949571e-01 2.20518648e-01 0.00000000e+00]
 [7.74737809e-01 0.00000000e+00 3.17382812e-06]
 [7.71931012e-01 2.32399540e+00 4.49620816e-03]]
V=
 [[0.         1.10055886 0.65365381 0.         1.08167464]
 [0.40238263 0.         0.20306013 0.02030649 0.13511108]
 [2.21765711 4.56852448 1.32563828 8.58271643 0.        ]]
UV=
 [[8.23476850e-01 3.92459465e-01 4.33303220e-01 7.69242120e-01
  2.12536766e-01]
 [1.13628490e+00 3.75822009e-01 7.96631357e-01 5.73433330e-02
  7.50912402e-01]
 [8.87328739e-02 1.08619610e+00 6.89901896e-01 4.47796037e-03
  1.09735283e+00]
 [7.03846249e-06 8.52659060e-01 5.06414531e-01 2.72400668e-05
  8.38014237e-01]
 [9.45106433e-01 8.70096552e-01 9.82446798e-01 8.57818759e-02
  1.14897573e+00]]


# calling iPALM

In [526]:
# A = np.random.randint(2,size=(50, 100)).astype("float")
# print(A)

In [714]:
# def factorize(A,k,l1reg,l2reg,regularization_rate,tol):



n,m=A.shape
print(n,m)
k=4
# U = np.random.randint(2,size=(n, k)).astype("float")
# V = np.random.randint(2,size=(k, m)).astype("float")
U,V=factorize_ipalm(fn,A,U,V,0.01,1000,0.01,0.1)
# rounding(fn,A,U,V)
A.shape

5 5


(5, 5)

In [715]:
print(U.shape,V.shape)
print("A=\n",A)
print("U=\n",U)
print("V=\n",V)
print("UV=\n",np.dot(U,V))

(5, 3) (3, 5)
A=
 [[1 0 0 1 0]
 [1 0 1 0 1]
 [0 1 1 0 1]
 [0 1 0 0 1]
 [1 1 1 0 1]]
U=
 [[0.00000000e+00 1.56698820e+00 9.13712761e-02]
 [2.91318638e-01 2.58161664e+00 0.00000000e+00]
 [7.61375358e-01 2.01100513e-01 8.70054877e-04]
 [5.92323424e-01 0.00000000e+00 0.00000000e+00]
 [6.18731221e-01 2.15073244e+00 4.38602636e-06]]
V=
 [[0.         1.40752811 0.88403464 0.         1.41404133]
 [0.44333563 0.         0.19526524 0.0508673  0.12841477]
 [1.30175885 4.17464621 1.83509055 6.03634406 0.        ]]
UV=
 [[0.81364507 0.38144275 0.4736529  0.63125692 0.20122443]
 [1.14452265 0.41003917 0.76163577 0.13131987 0.74345431]
 [0.09028762 1.07528939 0.71394676 0.01548139 1.1024405 ]
 [0.         0.83371187 0.52363443 0.         0.8375698 ]
 [0.95350204 0.87089989 0.96695118 0.10942843 1.15109733]]


In [529]:
np.linalg.norm(A - U @ V)**2

7.452991293309875

In [499]:
rounding(fn,A,U,V)

In [500]:
print("A=\n",A)
print("U=\n",U)
print("V=\n",V)
print("UV=\n",np.dot(U,V))

A=
 [[1 0 0 1 0]
 [1 0 1 0 1]
 [0 1 1 0 1]
 [0 1 0 0 1]
 [1 1 1 0 1]]
U=
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
V=
 [[0. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1.]]
UV=
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [530]:
np.dot(U,V)

array([[0.01309341, 0.49616905, 0.03156523, 0.11652415, 0.48537548],
       [0.6677754 , 0.33736096, 0.58725276, 0.07922844, 0.33002207],
       [0.52960952, 0.41112089, 0.47167736, 0.09655079, 0.40217745],
       [0.04981226, 1.88761309, 0.12008598, 0.44330155, 1.8465503 ],
       [1.05435243, 0.17647553, 0.91286028, 0.04144487, 0.17263651]])

In [47]:
import numpy as np

def matrix_factorization_PALM(fn,R, k, lambda_1=0.01,lambda_2=0.02, max_iter=1000, tol=1e-4):
    """
    Matrix factorization using Proximal Alternating Linear Minimization (PALM)
    
    Parameters:
        R (np.ndarray): Input matrix to factorize
        k (int): Number of latent factors
        max_iter (int): Maximum number of iterations
        tol (float): Tolerance for convergence
        lambda_ (float): Regularization term
        
    Returns:
        (np.ndarray, np.ndarray): The two factor matrices
    """
    
    m, n = R.shape
    X = np.random.rand(m, k)
    Y = np.random.rand(k, n)
    
    # Set the step size
    eta = 1 / (np.linalg.norm(X, 2) * np.linalg.norm(Y, 2))
    fn0=copy.deepcopy(fn)
    for i in range(max_iter):
        fn.l2reg = fn0.l2reg * regularization_rate(t - 1)
#         print(i)
    #         X_prev = X.copy()
    #         Y_prev = Y.copy()

        # Minimize with respect to X

#         for j in range(m):
        YYt = Y @ Y.T
        RYt = R @ Y.T
        L = max(np.linalg.norm(YYt), 1e-4)
        eta = 1/(1.1*L)
        reg=regularizer_elbmf(X,lambda_1,lambda_2)
        grad_X = X @ YYt - RYt +reg * np.sign(X)
#         print(grad_X)
#     * np.sign(R)
#         +reg * np.sign(X)
        X = X-grad_X * eta


        # Minimize with respect to Y
#         for j in range(n):
        XtX = X.T @ X
        RtX = R.T @ X
        L = max(np.linalg.norm(XtX), 1e-4)
        eta = 1/(1.1*L)
        reg=regularizer_elbmf(Y.T,lambda_1,lambda_2)
        grad_Yt = Y.T @ XtX - RtX +reg * np.sign(Y.T)
#     * np.sign(R)
#         +reg * np.sign(Y.T)
        Yt = Y.T-grad_Yt *eta


        
#         ell = np.linalg.norm(R - X @ Y)**2
#         ell0 = ell
        # Check for convergence
    #         if (abs(ell - ell0) < tol):
    #             break   
    return X, Y


In [48]:
R = np.random.randint(6, size=(100,400), dtype=np.int8)
# R=np.array([
#    [1,0,0,4,5,0],
#    [0,0,3,3,0,4],
#    [0,4,0,2,0,3],
#    [2,1,5,0,1,3],
#   ])


print(R)
print("")


K = 10


nP, nQ = matrix_factorization_PALM(R,K)

nR = np.dot(nP, nQ)
print(nR)
print("")
print(np.linalg.norm(R-nR)**2)

[[4 1 3 ... 2 4 1]
 [2 5 0 ... 4 1 2]
 [3 1 1 ... 2 0 2]
 ...
 [2 5 5 ... 2 4 5]
 [0 3 1 ... 5 5 4]
 [0 3 1 ... 5 2 0]]



TypeError: matrix_factorization_PALM() missing 1 required positional argument: 'k'

# Dataset

In [ ]:
# Reads the movie lens data and creates a matrix out of it 

import numpy as np
import pandas as pd
ratings = pd.read_csv("/home/cris-musa/Documents/Dataset/ratings.csv")
movies = pd.read_csv("/home/cris-musa/Documents/Dataset/movies.csv")
# print(ratings)
# print(movies)

R = np.zeros(shape=(610,193609),dtype=float)
for i in range(100835):
    user = ratings["userId"][i]-1
    movie = ratings["movieId"][i]-1
    R[user][movie]=ratings["rating"][i]
print(R)
print(np.sign(R))

In [ ]:
print(R)
nP, nQ = matrix_factorization_PALM(R,40)
nR = np.dot(nP, nQ)
print(nR/5)
print("")
print(np.linalg.norm(R-nR)**2)

In [ ]:
# R = np.random.randint(6, size=(100,400), dtype=np.int8)
R=np.array([
   [1,0,0,4,5,0],
   [0,0,3,3,0,4],
   [0,4,0,2,0,3],
   [2,1,5,0,1,3],
  ])


print(R)
print("")


K = 3


nP, nQ = matrix_factorization_PALM(R,K)

nR = np.dot(nP, nQ)
print(nR)
print("")
print(np.linalg.norm(R-nR)**2)


In [ ]:
A = np.array([
    [1,2,3,4],
    [0,1,0,5],
    [4,0,1,3],
])
B = np.array([
    [1,1,1,1],
    [1,1,1,1],
    [1,1,1,1],
])

A*5

In [ ]:
import numpy as np

def ALS(A, k, max_iter=100, tol=1e-4):
    m, n = A.shape
    U = np.random.rand(m, k)
    V = np.random.rand(n, k)
    for t in range(max_iter):
        # Update U
        V_T_V = np.dot(V.T, V)
        A_V = np.dot(A, V)
        for i in range(m):
            u_i = np.dot(A_V[i, :], np.linalg.inv(V_T_V))
            U[i, :] = u_i / np.linalg.norm(u_i)
        
        # Update V
        U_T_U = np.dot(U.T, U)
        A_U = np.dot(A.T, U)
        for j in range(n):
            v_j = np.dot(A_U[j, :], np.linalg.inv(U_T_U))
            V[j, :] = v_j / np.linalg.norm(v_j)
        
        # Check for convergence
        if np.linalg.norm(A - np.dot(U, V.T)) < tol:
            break
    return U, V


In [ ]:
# R = np.random.randint(6, size=(100,400), dtype=np.int8)
R=np.array([
   [1,0,0,4,5,0],
   [0,0,3,3,0,4],
   [0,4,0,2,0,3],
   [2,0,5,0,1,3],
  ])


print(R)
print("")


K = 3


nP, nQ = ALS(R,K)

nR = np.dot(nP, nQ.T)
print(nR)
print("")
print(np.linalg.norm(R-nR)**2)